# Visualize inversion results

Use this notebook to visualize some standard inversion results.

See `utils.py` for the code to generate plots and count observations in the region of interest (imported below).

Notebook can be accessed via AWS following these instructions: https://docs.aws.amazon.com/dlami/latest/devguide/setup-jupyter.html)

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import sys
import os
import yaml
import pickle as pickle
import pandas as pd
import xesmf as xe
import cartopy
import cartopy.crs as ccrs
import colorcet as cc
from shapely.geometry.polygon import Polygon
import shapely.ops as ops
from functools import partial
import pyproj

from utils import (
    plot_field,
    load_obj,
    sum_total_emissions,
    count_obs_in_mask,
    get_posterior_emissions,
    get_mean_emissions,
    plot_ensemble,
    plot_hyperparameter_analysis,
    export_visualization_outputs,
)

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

ModuleNotFoundError: No module named 'utils'

## Setup

In [ ]:
# Read the configuration file
config = yaml.load(
    open("/home/ubuntu/integrated_methane_inversion/config.yml"), Loader=yaml.FullLoader
)

In [ ]:
# Paths to prior emissions, inversion results, GEOS/satellite data, posterior simulation
# update prior prefix and period based on whether using kalman mode
if config["KalmanMode"]:
    prior_prefix = "./../.."
    period = int(os.getcwd().split("kf_inversions/period")[-1])  # get current period
    periods_df = pd.read_csv("./../../periods.csv")
    start_date = periods_df.iloc[period - 1, 0]
    end_date = periods_df.iloc[period - 1, 1]
    prior_sf_pth = f"{prior_prefix}/archive_sf/prior_sf_period{period}.nc"
else:
    prior_prefix = "./.."
    start_date = config["StartDate"]
    end_date = config["EndDate"]

prior_cache = f"{prior_prefix}/hemco_prior_emis/OutputDir/"


satdat_dir = (
    "./data_converted_prior" if config["LognormalErrors"] else "./data_converted"
)
results_pth = (
    "./gridded_posterior_ln.nc"
    if config["LognormalErrors"]
    else "./gridded_posterior.nc"
)
inversion_result_path = (
    "./inversion_result_ln.nc" if config["LognormalErrors"] else "./inversion_result.nc"
)
posterior_dir = "./data_converted_posterior"
visualization_dir = (
    "./data_visualization_prior"
    if config["LognormalErrors"]
    else "./data_visualization"
)
posterior_viz_dir = "./data_visualization_posterior"

plot_save_path = ("./output" if not config["KalmanMode"] else f"../output/period_{period}_plots")

In [ ]:
# Open the state vector file
if config["KalmanMode"] and config["DynamicKFClustering"]:
    state_vector_filepath = f"./../../archive_sv/StateVector_{period}.nc"
else:
    state_vector_filepath = "./../StateVector.nc"

state_vector = xr.open_dataset(state_vector_filepath)
state_vector_labels = state_vector["StateVector"]

# Identify the last element of the region of interest
last_ROI_element = int(
    np.nanmax(state_vector_labels.values) - config["nBufferClusters"]
)

# Define mask for region of interest
mask = state_vector_labels <= last_ROI_element

In [ ]:
# Set latitude/longitude bounds for plots

# Trim 1-2.5 degrees to remove GEOS-Chem buffer zone
if config["Res"] == "0.25x0.3125":
    degx = 4 * 0.3125
    degy = 4 * 0.25
elif config["Res"] == "0.5x0.625":
    degx = 4 * 0.625
    degy = 4 * 0.5
elif config["Res"] == "2.0x2.5":
    degx = 4 * 2.5
    degy = 4 * 2.0

lon_bounds = [
    np.min(state_vector.lon.values) + degx,
    np.max(state_vector.lon.values) - degx,
]
lat_bounds = [
    np.min(state_vector.lat.values) + degy,
    np.max(state_vector.lat.values) - degy,
]

## State vector

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.rcParams.update({"font.size": 16})
ax = fig.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})
num_colors = state_vector_labels.where(mask).max().item()
sv_cmap = matplotlib.colors.ListedColormap(np.random.rand(int(num_colors), 3))

plot_field(
    ax,
    state_vector_labels,
    cmap=sv_cmap,
    title="State vector elements",
    cbar_label="Element Id",
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

## Prior and posterior emissions in the region of interest

In [ ]:
# Prior emissions
prior_ds = get_mean_emissions(start_date, end_date, prior_cache)
prior = prior_ds["EmisCH4_Total"]

if config["KalmanMode"]:
    # properly apply nudged sfs to prior in Kalman mode
    prior_sf = xr.open_dataset(prior_sf_pth)
    prior_ds = get_posterior_emissions(prior_ds, prior_sf)
    prior = prior_ds["EmisCH4_Total"]

# Optimized scale factors
scale_ds = xr.open_dataset(results_pth)
scale = scale_ds["ScaleFactor"]

# ensemble results
ens_scale_ds = xr.open_dataset(results_pth.replace(".nc", "_ensemble.nc"))
ens_inv_result = xr.open_dataset(inversion_result_path.replace(".nc", "_ensemble.nc"))

# Posterior emissions
posterior_ds = get_posterior_emissions(prior_ds, scale_ds)
posterior = posterior_ds["EmisCH4_Total"]

In [ ]:
# Total emissions in the region of interest
areas = prior_ds["AREA"]

total_prior_emissions = sum_total_emissions(prior, areas, mask)
total_posterior_emissions = sum_total_emissions(posterior, areas, mask)

print("Prior         emissions :", total_prior_emissions, "Tg/y")
print(
    "Posterior     emissions :",
    total_posterior_emissions,
    "Tg/y (mean xhat)",
)

# Plot the ensemble of posterior emissions
# and calculate the mean and standard deviation
# Note: update the exclude_members list to exclude ensemble members from the analysis
# integer list of ensemble members to exclude from analysis
exclude_members = []  # eg. [0, 4, 5]
ensemble_members = ens_scale_ds.ensemble.values
ensemble_members = [num for num in ensemble_members if num not in exclude_members]
num_ensemble_members = len(ensemble_members)
default_ensemble_member = int(ens_scale_ds.attrs["default_member_index"])
if num_ensemble_members > 1:
    ensemble_posteriors = []
    for imem in ensemble_members:
        ensemble_posteriors.append(
            get_posterior_emissions(
                prior_ds, ens_scale_ds.sel(ensemble=imem)["ScaleFactor"]
            )
        )

    ens_totals_posterior = np.array(
        [
            sum_total_emissions(item["EmisCH4_Total"], areas, mask)
            for item in ensemble_posteriors
        ]
    )
    ens_totals_min = np.min(ens_totals_posterior)
    ens_totals_max = np.max(ens_totals_posterior)
    print("Ensemble range:", f" {ens_totals_min}-{ens_totals_max} Tg/yr")

    # Plot the totals
    include_default = not default_ensemble_member in exclude_members
    fig = plt.figure(figsize=(8, 8))
    plt.rcParams.update({"font.size": 14})
    ax = fig.subplots(1, 1)
    plot_ensemble(
        ax,
        ens_totals_posterior,
        total_prior_emissions,
        plot_save_path=plot_save_path,
    )

In [ ]:
# Plot sensitivity of total emissions to hyperparameters
if num_ensemble_members > 1:
    # Collect parameters with multiple values used in ensemble members
    params = [
        "prior_err",
        "obs_err",
        "gamma",
        "prior_err_bc",
        "prior_err_oh",
    ]
    if config["LognormalErrors"]:
        params.append("prior_err_buffer")
    params_dict = {
        key: ens_inv_result[key].values[ensemble_members]
        for key in params
        if len(np.unique(ens_inv_result[key].values[ensemble_members])) > 1
    }

    # Only proceed if there are axes to plot
    num_axes = len(params_dict.keys())
    if num_axes > 0:
        fig, axs = plt.subplots(num_axes, 1, figsize=(8, 3 * num_axes), squeeze=False)
        plt.rcParams.update({"font.size": 14})
        plot_hyperparameter_analysis(axs, ens_totals_posterior, params_dict)

In [ ]:
# Plot prior emissions
fig = plt.figure(figsize=(8, 8))
plt.rcParams.update({"font.size": 16})
ax = fig.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})

prior_kgkm2h = prior * (1000**2) * 60 * 60  # Units kg/km2/h

plot_field(
    ax,
    prior_kgkm2h,
    cmap=cc.cm.linear_kryw_5_100_c67_r,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    vmin=0,
    vmax=14,
    title="Prior emissions",
    cbar_label="Emissions (kg km$^{-2}$ h$^{-1}$)",
    only_ROI=True,
    state_vector_labels=state_vector_labels,
    last_ROI_element=last_ROI_element,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

In [ ]:
# Plot posterior emissions
fig = plt.figure(figsize=(8, 8))
plt.rcParams.update({"font.size": 16})
ax = fig.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})

posterior_kgkm2h = posterior * (1000**2) * 60 * 60  # Units kg/km2/h

plot_field(
    ax,
    posterior_kgkm2h,
    cmap=cc.cm.linear_kryw_5_100_c67_r,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    vmin=0,
    vmax=14,
    title="Posterior emissions",
    cbar_label="Emissions (kg km$^{-2}$ h$^{-1}$)",
    only_ROI=True,
    state_vector_labels=state_vector_labels,
    last_ROI_element=last_ROI_element,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

## Sectoral Emissions

In [ ]:
# Extract sector names
sectors = [
    var
    for var in list(posterior_ds.keys())
    if "EmisCH4" in var and not ("Total" in var or "Excl" in var)
]

# Calculate total emissions for each sector
prior_sector_vals = []
post_sector_vals = []
positive_sectors = []
for sector in sectors:
    post_val = sum_total_emissions(posterior_ds[sector], areas, mask)
    prior_val = sum_total_emissions(prior_ds[sector], areas, mask)
    if post_val > 0 or prior_val > 0:
        post_sector_vals.append(post_val)
        prior_sector_vals.append(prior_val)
        positive_sectors.append(sector.replace("EmisCH4_", ""))

# Combine the lists into tuples and sort them based on post_sector_vals
combined = list(zip(positive_sectors, prior_sector_vals, post_sector_vals))
combined_sorted = sorted(combined, key=lambda x: x[2])
positive_sectors, prior_sector_vals, post_sector_vals = zip(*combined_sorted)

# Plot bars for prior and posterior
fig = plt.figure(figsize=(10, 5))
ax = fig.subplots(1, 1)
bar_height = 0.35
ind = np.arange(len(positive_sectors))
bars2 = ax.barh(
    ind + bar_height / 2,
    post_sector_vals,
    bar_height,
    color="steelblue",
    label="Posterior Emissions",
)
bars1 = ax.barh(
    ind - bar_height / 2,
    prior_sector_vals,
    bar_height,
    color="goldenrod",
    label="Prior Emissions",
)

# Add labels, title, and legend
ax.set_xlabel("Emissions ($Tg\ a^{-1}$)")
ax.set_ylabel("Sector")
ax.set_title("Sectoral emissions")
ax.set_yticks(ind)
ax.set_yticklabels(positive_sectors)
ax.legend()

plt.savefig(f"{plot_save_path}/Sectoral_emissions.png", bbox_inches="tight")

## Scale factors

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.rcParams.update({"font.size": 16})
ax = fig.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})

plot_field(
    ax,
    scale,
    cmap="RdBu_r",
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    vmin=0,
    vmax=2,
    title="Scale factors",
    cbar_label="Scale factor",
    only_ROI=True,
    state_vector_labels=state_vector_labels,
    last_ROI_element=last_ROI_element,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

## Averaging kernel sensitivities

In [ ]:
S_post_grid = xr.open_dataset(results_pth)["S_post"]
A_grid = xr.open_dataset(results_pth)["A"]
avkern_ROI = A_grid.where(state_vector_labels <= last_ROI_element)

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.rcParams.update({"font.size": 16})
ax = fig.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})

plot_field(
    ax,
    avkern_ROI,
    cmap=cc.cm.CET_L19,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="Averaging kernel sensitivities",
    cbar_label="Sensitivity",
    only_ROI=True,
    state_vector_labels=state_vector_labels,
    last_ROI_element=last_ROI_element,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

In [ ]:
# ungridded inversion result is used to calculate DOFS using the trace of the averaging kernel
A_ROI = xr.open_dataset(inversion_result_path)["A"].values[:last_ROI_element, :last_ROI_element]
DOFS = np.trace(A_ROI)
print("DOFS =", DOFS)

## Open TROPOMI and GEOS-Chem columns

In [ ]:
# Get observed and GEOS-Chem-simulated TROPOMI columns
def aggregate_data(data_dir, data_posterior):
    files = np.sort(os.listdir(data_dir))
    lat = np.array([])
    lon = np.array([])
    tropomi = np.array([])
    geos_prior = np.array([])
    geos_posterior = np.array([])
    observation_count = np.array([])

    for f in files:
        # Get paths
        pth = os.path.join(data_dir, f)
        pth_posterior = os.path.join(data_posterior, f)
        # Load TROPOMI/GEOS-Chem and Jacobian matrix data from the .pkl file
        obj = load_obj(pth)
        # If there is no corresponding file in data_posterior dir, skip
        if not os.path.isfile(pth_posterior):
            continue
        obj_posterior = load_obj(pth_posterior)
        # If there aren't any TROPOMI observations on this day, skip
        if obj["obs_GC"].shape[0] == 0:
            continue
        # Otherwise, grab the TROPOMI/GEOS-Chem data
        obs_GC = obj["obs_GC"]
        obs_GC_posterior = obj_posterior["obs_GC"]
        # Only consider data within latitude and longitude bounds
        ind = np.where(
            (obs_GC[:, 2] >= lon_bounds[0])
            & (obs_GC[:, 2] <= lon_bounds[1])
            & (obs_GC[:, 3] >= lat_bounds[0])
            & (obs_GC[:, 3] <= lat_bounds[1])
        )
        if len(ind[0]) == 0:  # Skip if no data in bounds
            continue
        obs_GC = obs_GC[ind[0], :]  # TROPOMI and GEOS-Chem data within bounds
        obs_GC_posterior = obs_GC_posterior[ind[0], :]
        # Record lat, lon, tropomi ch4, and geos ch4
        lat = np.concatenate((lat, obs_GC[:, 3]))
        lon = np.concatenate((lon, obs_GC[:, 2]))
        tropomi = np.concatenate((tropomi, obs_GC[:, 0]))
        geos_prior = np.concatenate((geos_prior, obs_GC[:, 1]))
        observation_count = np.concatenate((observation_count, obs_GC[:, 4]))
        geos_posterior = np.concatenate((geos_posterior, obs_GC_posterior[:, 1]))

    df = pd.DataFrame()
    df["lat"] = lat
    df["lon"] = lon
    df["tropomi"] = tropomi
    df["geos_prior"] = geos_prior
    df["geos_posterior"] = geos_posterior
    df["diff_tropomi_prior"] = geos_prior - tropomi
    df["diff_tropomi_posterior"] = geos_posterior - tropomi
    df["observation_count"] = observation_count

    return df


superobs_df = aggregate_data(satdat_dir, posterior_dir)
visualization_df = aggregate_data(visualization_dir, posterior_viz_dir)
n_obs = len(superobs_df["tropomi"])

print(
    f'Found {n_obs} super-observations in the domain, representing {np.sum(superobs_df["observation_count"]).round(0)} TROPOMI observations.'
)
superobs_df.head()

Note: This observation count is for the lat/lon bounds defined in input cell 5.

In [ ]:
# calculate some statistics for the prior
prior_std = np.round(np.std(superobs_df["diff_tropomi_prior"]), 2)
prior_bias = np.round(
    np.average(
        superobs_df["diff_tropomi_prior"], weights=superobs_df["observation_count"]
    ),
    2,
)
prior_RMSE = np.round(
    np.sqrt(
        np.average(
            superobs_df["diff_tropomi_prior"] ** 2,
            weights=superobs_df["observation_count"],
        )
    ),
    2,
)

# and the posterior
posterior_std = np.round(np.std(superobs_df["diff_tropomi_posterior"]), 2)
posterior_bias = np.round(
    np.average(
        superobs_df["diff_tropomi_posterior"], weights=superobs_df["observation_count"]
    ),
    2,
)
posterior_RMSE = np.round(
    np.sqrt(
        np.average(
            superobs_df["diff_tropomi_posterior"] ** 2,
            weights=superobs_df["observation_count"],
        )
    ),
    2,
)

# Print some error statistics
print(f"Bias in prior     : {prior_bias} +/- {prior_std} ppb")
print(f"RMSE prior        : {prior_RMSE} ppb")
print(f"Bias in posterior : {posterior_bias} +/- {posterior_std} ppb")
print(f"RMSE posterior    : {posterior_RMSE} ppb")

## Count observations within the region of interest

In [ ]:
print("Found", count_obs_in_mask(mask, superobs_df), "super-observations within the region of interest")

## Compare TROPOMI and GEOS-Chem columns

### Comparison at 0.1 x 0.1 resolution

In [ ]:
# Simple averaging scheme to grid the XCH4 data at 0.1 x 0.1 resolution
df_copy = visualization_df.copy()  # save for later
visualization_df["lat"] = np.round(visualization_df["lat"], 1)
visualization_df["lon"] = np.round(visualization_df["lon"], 1)
visualization_df = visualization_df.groupby(["lat", "lon"]).mean()
ds = visualization_df.to_xarray()

# simple function to find the dynamic range for colorbar
dynamic_range = lambda vals: (
    np.round(np.nanmedian(vals) / 25.0) * 25 - 25,
    np.round(np.nanmedian(vals) / 25.0) * 25 + 25,
)

In [ ]:
# Mean TROPOMI XCH4 columns on 0.1 x 0.1 grid
fig = plt.figure(figsize=(8, 8))
ax = fig.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})

xch4_min, xch4_max = dynamic_range(ds["tropomi"].values)
plot_field(
    ax,
    ds["tropomi"],
    cmap="Spectral_r",
    vmin=xch4_min,
    vmax=xch4_max,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="TROPOMI $X_{CH4}$",
    clean_title="TROPOMI XCH4",
    cbar_label="Column mixing ratio (ppb)",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

In [ ]:
# Mean prior and posterior GEOS-Chem XCH4 columns on 0.1 x 0.1 grid
fig = plt.figure(figsize=(25, 8))
ax1, ax2 = fig.subplots(1, 2, subplot_kw={"projection": ccrs.PlateCarree()})

plot_field(
    ax1,
    ds["geos_prior"],
    cmap="Spectral_r",
    vmin=xch4_min,
    vmax=xch4_max,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="GEOS-Chem $X_{CH4}$ (prior simulation)",
    clean_title="GEOS-Chem XCH4 (prior)",
    cbar_label="Dry column mixing ratio (ppb)",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
)

plot_field(
    ax2,
    ds["geos_posterior"],
    cmap="Spectral_r",
    vmin=xch4_min,
    vmax=xch4_max,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="GEOS-Chem $X_{CH4}$ (posterior simulation)",
    clean_title="GEOS-Chem XCH4 (Prior&Posterior)",
    cbar_label="Dry column mixing ratio (ppb)",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

In [ ]:
# Plot differences between GEOS-Chem and TROPOMI XCH4
fig = plt.figure(figsize=(25, 8))
ax1, ax2 = fig.subplots(1, 2, subplot_kw={"projection": ccrs.PlateCarree()})

plot_field(
    ax1,
    ds["diff_tropomi_prior"],
    cmap="RdBu_r",
    vmin=-40,
    vmax=40,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="Prior $-$ TROPOMI",
    clean_title="Prior-TROPOMI",
    cbar_label="ppb",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
)

plot_field(
    ax2,
    ds["diff_tropomi_posterior"],
    cmap="RdBu_r",
    vmin=-40,
    vmax=40,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="Posterior $-$ TROPOMI",
    clean_title="(Prior&Posterior)-TROPOMI",
    cbar_label="ppb",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

In [ ]:
# Plot differences between posterior and prior simulated XCH4
fig = plt.figure(figsize=(8, 8))
ax = fig.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})

ds['geos_diff'] = ds["geos_posterior"] - ds["geos_prior"]

plot_field(
    ax,
    ds['geos_diff'],
    cmap="seismic",
    vmin=-np.nanmax(ds['geos_diff']),
    vmax=np.nanmax(ds['geos_diff']),
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="$\Delta X_{CH4}$ (Posterior $-$ Prior)",
    clean_title="Delta XCH4 (Posterior-Prior)",
    cbar_label="ppb",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

### Comparison at 0.25 x 0.3125 resolution

In [ ]:
# Simple averaging scheme to grid the XCH4 data at 0.1 x 0.1 resolution
df_copy = superobs_df.copy()  # save for later
superobs_df["lat"] = np.round(superobs_df["lat"], 1)
superobs_df["lon"] = np.round(superobs_df["lon"], 1)
superobs_df = superobs_df.groupby(["lat", "lon"]).mean()
superobs_df = superobs_df.reset_index()

In [ ]:
# Fill in any missing lat/lon coordinates at .1 degree spacing
new_lat_values = np.arange(ds["lat"][0], ds["lat"][-1] + 0.1, 0.1)
new_lon_values = np.arange(ds["lon"][0], ds["lon"][-1] + 0.1, 0.1)

data = []
# Iterate through latitude and longitude combinations
for lat in new_lat_values:
    for lon in new_lon_values:
        data.append({"lat": lat, "lon": lon, "value": None})

# Create a new DataFrame from the data
df_temp = pd.DataFrame(data)

# ensure matching sigfigs with superobs_df
df_temp["lat"] = np.round(df_temp["lat"], 1)
df_temp["lon"] = np.round(df_temp["lon"], 1)

# merge the two dataframes and drop the filler column
# then convert to xarray dataset
merged_df = pd.merge(df_temp, superobs_df, on=["lat", "lon"], how="left").drop(
    columns=["value"]
)
ds = xr.Dataset.from_dataframe(merged_df.set_index(["lat", "lon"]))

In [ ]:
# calculate the grid bounds for .1x.1 grid
lat_b = np.arange(ds["lat"][0] - 0.05, ds["lat"][-1] + 0.1, 0.1)
lon_b = np.arange(ds["lon"][0] - 0.05, ds["lon"][-1] + 0.1, 0.1)
ds = ds.assign_coords(lon_b=("lon_b", lon_b))
ds = ds.assign_coords(lat_b=("lat_b", lat_b))
ds["mask"] = xr.where(~np.isnan(ds["tropomi"]), 1, 0)

In [ ]:
# Global 0.25 x 0.3125 grid
reference_lat_grid = np.arange(-90, 90 + 0.25, 0.25)
reference_lon_grid = np.arange(-180, 180 + 0.3125, 0.3125)

# Find closest reference coordinates to selected lat/lon bounds
lat_min = reference_lat_grid[np.abs(reference_lat_grid - lat_bounds[0]).argmin()]
lon_min = reference_lon_grid[np.abs(reference_lon_grid - lon_bounds[0]).argmin()]
lat_max = reference_lat_grid[np.abs(reference_lat_grid - lat_bounds[1]).argmin()]
lon_max = reference_lon_grid[np.abs(reference_lon_grid - lon_bounds[1]).argmin()]

# Create an xESMF regridder object to resample the data on the grid HEMCO expects
new_lat_grid = np.arange(lat_min, lat_max + 0.25, 0.25)
new_lon_grid = np.arange(lon_min, lon_max + 0.3125, 0.3125)
new_lat_b = np.arange(new_lat_grid[0] - 0.125, new_lat_grid[-1] + 0.25, 0.25)
new_lon_b = np.arange(new_lon_grid[0] - 0.15625, new_lon_grid[-1] + 0.3125, 0.3125)
ds_out = xr.Dataset(
    {
        "lat": (["lat"], new_lat_grid),
        "lon": (["lon"], new_lon_grid),
        "lat_b": (["lat_b"], new_lat_b),
        "lon_b": (["lon_b"], new_lon_b),
    }
)

regridder = xe.Regridder(ds, ds_out, "conservative_normed")

In [ ]:
# Regrid the data
ds_regrid = regridder(ds)


In [ ]:
# Re-plot differences between GEOS-Chem and TROPOMI XCH4
fig = plt.figure(figsize=(25, 8))
ax1, ax2 = fig.subplots(1, 2, subplot_kw={"projection": ccrs.PlateCarree()})

plot_field(
    ax1,
    ds_regrid["diff_tropomi_prior"],
    cmap="RdBu_r",
    vmin=-25,
    vmax=25,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="Prior $-$ TROPOMI",
    clean_title="Prior-TROPOMI (regridded)",
    cbar_label="ppb",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
)

plot_field(
    ax2,
    ds_regrid["diff_tropomi_posterior"],
    cmap="RdBu_r",
    vmin=-25,
    vmax=25,
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="Posterior $-$ TROPOMI",
    clean_title="(Prior&Posterior)-TROPOMI (regridded)",
    cbar_label="ppb",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

In [ ]:
# Re-plot differences between posterior and prior simulated XCH4
fig = plt.figure(figsize=(8, 8))
ax = fig.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})

ds_regrid['geos_diff'] = ds_regrid["geos_posterior"] - ds_regrid["geos_prior"]

plot_field(
    ax,
    ds_regrid["geos_diff"],
    cmap="seismic",
    vmin=-np.nanmax(ds_regrid["geos_diff"]),
    vmax=np.nanmax(ds_regrid["geos_diff"]),
    lon_bounds=lon_bounds,
    lat_bounds=lat_bounds,
    title="Posterior $-$ Prior",
    clean_title="Posterior-Prior (regridded)",
    cbar_label="ppb",
    mask=mask,
    only_ROI=False,
    is_regional=config["isRegional"],
    save_path=plot_save_path
)

In [ ]:
# Export visualization outputs
_ = export_visualization_outputs(
    plot_save_path=plot_save_path,
    start_date=start_date,
    end_date=end_date,
    total_prior_emissions=total_prior_emissions,
    total_posterior_emissions=total_posterior_emissions,
    DOFS=DOFS,
    prior_bias=prior_bias,
    posterior_bias=posterior_bias,
    prior_std=prior_std,
    posterior_std=posterior_std,
    prior_RMSE=prior_RMSE,
    posterior_RMSE=posterior_RMSE,
    df_for_count=df_copy,
    mask=mask,
    combined_sorted=combined_sorted,
    prior_ds=prior_ds,
    posterior_ds=posterior_ds,
    areas=areas,
    num_ensemble_members=num_ensemble_members,
    ens_totals_posterior=(ens_totals_posterior if num_ensemble_members > 1 else None),
    ens_inv_result=(ens_inv_result if num_ensemble_members > 1 else None),
    ens_scale_ds=(ens_scale_ds if num_ensemble_members > 1 else None),
    state_vector_labels=state_vector_labels,
    scale=scale,
    avkern_ROI=avkern_ROI,
    ds_obs=ds,
    ds_obs_regridded=ds_regrid,
)


NameError: name 'start_date' is not defined

----